# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv = "../python-api-challenge/output/cities.csv"

city_weather = pd.read_csv(csv)

city_weather.head()

,City,City.1,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.0970,-79.2353,63.99,47,83,2.91,US,2022-08-10
1,1,bluff,-46.6000,168.3333,44.80,57,8,10.22,NZ,2022-08-10
2,2,busselton,-33.6500,115.3333,55.62,75,89,3.62,AU,2022-08-10
3,3,vaini,-21.2000,-175.2000,80.76,89,75,17.27,TO,2022-08-10
4,4,rikitea,-23.1203,-134.9692,73.24,73,100,22.44,PF,2022-08-10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_weather[['Lat', 'Lon']]
weights = city_weather['Humidity']

layout = {
    'width':'100%',
    'height':'400px',
    'border':'1px sold black',
    'padding': '1px'
}
fig = gmaps.figure(layout=layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100, point_radius= 1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(border='1px sold black', height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weather = city_weather.loc[
    (city_weather['Max Temp']> 70) &
    (city_weather['Max Temp']<90) &
    (city_weather['Humidity']<70) &
    (city_weather['Cloudiness']<50) &
    (city_weather['Wind Speed']<20) &
    (city_weather['Country'] != 'RU')
]
ideal_weather.reset_index()
ideal_weather.head()

,City,City.1,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,kapaa,22.0752,-159.3190,87.78,69,20,18.41,US,2022-08-10
37,37,honiara,-9.4333,159.9500,85.89,66,42,7.05,SB,2022-08-10
40,40,tevaitoa,-16.7833,-151.5000,78.12,69,0,5.99,PF,2022-08-10
43,43,faanui,-16.4833,-151.7500,78.30,69,0,6.76,PF,2022-08-10
68,68,kloulklubed,7.0419,134.2556,84.70,69,11,1.72,PW,2022-08-10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather[['City', 'Lat', 'Lon', 'Country']].copy()
hotel_df['Hotel Name'] = np.nan
hotel_df = hotel_df.rename(
    columns={
        'Lon':'Lng'
    }
)
hotel_df.head()

,City,Lat,Lng,Country,Hotel Name
22,22,22.0752,-159.3190,US,NaN
37,37,-9.4333,159.9500,SB,NaN
40,40,-16.7833,-151.5000,PF,NaN
43,43,-16.4833,-151.7500,PF,NaN
68,68,7.0419,134.2556,PW,NaN


In [8]:
ideal_weather.head()

,City,City.1,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,kapaa,22.0752,-159.3190,87.78,69,20,18.41,US,2022-08-10
37,37,honiara,-9.4333,159.9500,85.89,66,42,7.05,SB,2022-08-10
40,40,tevaitoa,-16.7833,-151.5000,78.12,69,0,5.99,PF,2022-08-10
43,43,faanui,-16.4833,-151.7500,78.30,69,0,6.76,PF,2022-08-10
68,68,kloulklubed,7.0419,134.2556,84.70,69,11,1.72,PW,2022-08-10


In [9]:
# I need a place to sleep after all of my fun
target_search = 'hotel'
target_radius = 5000
target_type = 'lodging'

params = {
    'keyword': target_search,
    'radius': target_radius,
    'type': target_type,
    'key': g_key
}
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df[0:10].iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat}, {lng}"

    hotel_data = requests.get(url, params).json()

# Save the first hotel name for each city
    try:
        hotel_name = hotel_data['results'][0]['name']
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        print(f"City '{row['City']}' found: The nearest hotel is {hotel_name} ")
        print("----------------------")
    except:
        print(f"Couldn't find any information for city '{row['City']}' . . . skipping.") 
        print("----------------------")

City '22' found: The nearest hotel is Kauai Shores Hotel 
----------------------
City '37' found: The nearest hotel is Heritage Park Hotel 
----------------------
City '40' found: The nearest hotel is Hôtel Raiatea Lodge 
----------------------
City '43' found: The nearest hotel is InterContinental Bora Bora Resort Thalasso Spa 
----------------------
City '68' found: The nearest hotel is The Adventures Inn-Peleliu 
----------------------
Couldn't find any information for city '87' . . . skipping.
----------------------
Couldn't find any information for city '108' . . . skipping.
----------------------
City '140' found: The nearest hotel is Hampton Inn & Suites Oahu/Kapolei 
----------------------
City '148' found: The nearest hotel is Vanimo Beach Hotel 
----------------------
City '174' found: The nearest hotel is Hôtel El Bey Constantine 
----------------------


In [10]:
hotel_df.dropna(how="any", inplace=True)
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Lat,Lng,Country,Hotel Name
0,22,22.0752,-159.3190,US,Kauai Shores Hotel
1,37,-9.4333,159.9500,SB,Heritage Park Hotel
2,40,-16.7833,-151.5000,PF,Hôtel Raiatea Lodge
3,43,-16.4833,-151.7500,PF,InterContinental Bora Bora Resort Thalasso Spa
4,68,7.0419,134.2556,PW,The Adventures Inn-Peleliu
5,140,21.3469,-158.0858,US,Hampton Inn & Suites Oahu/Kapolei
6,148,-2.6741,141.3028,PG,Vanimo Beach Hotel
7,174,36.3650,6.6147,DZ,Hôtel El Bey Constantine


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px sold black', height='400px', padding='1px', width='100%'))